Installing all the necessary needed to clean the data set

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from scipy import stats
import plotly.express as px
import re

In [315]:
import warnings
import seaborn as sns

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

I have now uploaded the original uncleaned file in jupyter and using pandas to read, access and edit the file

In [318]:
df = pd.read_csv('Uncleaned_DS_jobs.csv')
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


Droping the index column since i felt its unnecessary to have this column 

In [321]:
df.drop('index', inplace=True, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          672 non-null    object 
 1   Salary Estimate    672 non-null    object 
 2   Job Description    672 non-null    object 
 3   Rating             672 non-null    float64
 4   Company Name       672 non-null    object 
 5   Location           672 non-null    object 
 6   Headquarters       672 non-null    object 
 7   Size               672 non-null    object 
 8   Founded            672 non-null    int64  
 9   Type of ownership  672 non-null    object 
 10  Industry           672 non-null    object 
 11  Sector             672 non-null    object 
 12  Revenue            672 non-null    object 
 13  Competitors        672 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 73.6+ KB


# Data Cleaning Part

# My Approach to Extracting Job Description Information

1. **Skills Extraction**:
   - When I looked at the job descriptions, I realized it’s important to know which specific technical skills employers are asking for. So, I wrote a function to pick out key skills like Python, SQL, and Java.

2. **Education Extraction**:
   - I noticed that education levels vary across job listings, so I decided to extract qualifications like Bachelor’s, Master’s, and PhD to get a clearer picture of what’s required for each role.

3. **Experience Extraction**:
   - I saw that years of experience are often mentioned, but in different ways. To standardize this, I created a function to pull out the number of years required, whether it’s written as "3 years" or "5+ years".

4. **Application**:
   - By applying these functions to the job descriptions, I’m transforming unstructured text into organized data. This makes it much easier to analyze trends and requirements across different job listings.

In [325]:

def extract_skills(description):
    skills = ['Python', 'R', 'SQL', 'Java', 'C++', 'Scala', 'Julia']
    return [skill for skill in skills if skill.lower() in description.lower()]

def extract_education(description):
    education_levels = ['Bachelor', 'Master', 'PhD', 'Doctorate']
    return [level for level in education_levels if level.lower() in description.lower()]

def extract_experience(description):
    experience = re.search(r'(\d+)\+?\s*(?:year|yr)s?', description, re.IGNORECASE)
    return int(experience.group(1)) if experience else np.nan

df['Skills'] = df['Job Description'].apply(extract_skills)   
df['Education'] = df['Job Description'].apply(extract_education) 
df['Years_Experience'] = df['Job Description'].apply(extract_experience) 

In [326]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Skills,Education,Years_Experience
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",[R],"[Bachelor, Master]",NaN
1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1,"[R, SQL]",[Bachelor],15.0
2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1,"[Python, R, C++]",[PhD],NaN
3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...","[Python, R, SQL]",[Bachelor],NaN
4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee","[Python, R, SQL]",[],NaN


# My Approach to Processing Salary Estimates

1. **Initial Cleanup**:
   - I noticed that the 'Salary Estimate' column often includes extra information in parentheses. To simplify it, I split the string at the first '(', keeping only the part before it.

2. **Removing Unwanted Characters**:
   - To focus on the numerical values, I removed the dollar sign ('$') and the letter 'K' (which indicates thousands) from the salary estimates, also trimming any extra spaces.

3. **Splitting Salary into Min and Max**:
   - I split the cleaned salary estimates into two new columns: 'Salary Min' and 'Salary Max' using the '-' delimiter, so I could analyze the range of salaries more effectively.

4. **Type Conversion and Scaling**:
   - I converted these new columns from strings to integers and multiplied by 1000 to represent the full salary amounts.

5. **Calculating Average Salary**:
   - Finally, I calculated the average salary by taking the mean of 'Salary Min' and 'Salary Max', making it easier to assess the typical salary offered.

In [330]:
df['Salary Estimate'].unique()

array(['$137K-$171K (Glassdoor est.)', '$75K-$131K (Glassdoor est.)',
       '$79K-$131K (Glassdoor est.)', '$99K-$132K (Glassdoor est.)',
       '$90K-$109K (Glassdoor est.)', '$101K-$165K (Glassdoor est.)',
       '$56K-$97K (Glassdoor est.)', '$79K-$106K (Glassdoor est.)',
       '$71K-$123K (Glassdoor est.)', '$90K-$124K (Glassdoor est.)',
       '$91K-$150K (Glassdoor est.)', '$141K-$225K (Glassdoor est.)',
       '$145K-$225K(Employer est.)', '$79K-$147K (Glassdoor est.)',
       '$122K-$146K (Glassdoor est.)', '$112K-$116K (Glassdoor est.)',
       '$110K-$163K (Glassdoor est.)', '$124K-$198K (Glassdoor est.)',
       '$79K-$133K (Glassdoor est.)', '$69K-$116K (Glassdoor est.)',
       '$31K-$56K (Glassdoor est.)', '$95K-$119K (Glassdoor est.)',
       '$212K-$331K (Glassdoor est.)', '$66K-$112K (Glassdoor est.)',
       '$128K-$201K (Glassdoor est.)', '$138K-$158K (Glassdoor est.)',
       '$80K-$132K (Glassdoor est.)', '$87K-$141K (Glassdoor est.)',
       '$92K-$155K (Glassdo

In [332]:
df['Salary Estimate'] = df['Salary Estimate'].str.split('(', n=1).str[0]
df['Salary Estimate'] = df['Salary Estimate'].str.replace('$', '').str.strip()
df['Salary Estimate'] = df['Salary Estimate'].str.replace('K', '').str.strip()
df[['Salary Min', 'Salary Max']] = df['Salary Estimate'].str.split('-', expand=True)
df['Salary Min'] = df['Salary Min'].astype(int) * 1000
df['Salary Max'] = df['Salary Max'].astype(int) * 1000
df['avg_salary'] = (df['Salary Min'] + df['Salary Max']) // 2

In [334]:
df['Salary Estimate'].unique()

array(['137-171', '75-131', '79-131', '99-132', '90-109', '101-165',
       '56-97', '79-106', '71-123', '90-124', '91-150', '141-225',
       '145-225', '79-147', '122-146', '112-116', '110-163', '124-198',
       '79-133', '69-116', '31-56', '95-119', '212-331', '66-112',
       '128-201', '138-158', '80-132', '87-141', '92-155', '105-167'],
      dtype=object)

# My Approach to Cleaning Company Names

In [337]:
df[['Company Name']].sample(5)

,Company Name
601,ConsumerTrack\n3.2
583,MITRE\n3.3
225,Global Data Management Inc\n4.5
585,"Homology Medicines, Inc.\n4.4"
420,Robert Half\n3.5


In [339]:
df['Company Name'] = df['Company Name'].str.split('\n').str[0]

In [341]:
df[['Company Name']].sample(5)

,Company Name
253,Gallup
624,"ISYS Technologies, Inc."
15,Guzman & Griffin Technologies (GGTI)
109,SAIC
192,AstraZeneca


# My Approach to Cleaning Location

In [344]:
df[['Location']].sample(5)

,Location
528,"Washington, DC"
280,"Irvine, CA"
456,"Trumbull, CT"
191,"Santa Clara, CA"
138,"San Francisco, CA"


In [346]:
df['State'] = df['Location'].str.split(', ').str[-1]
df['City'] = df['Location'].str.split(', ').str[0]

In [348]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Revenue,Competitors,Skills,Education,Years_Experience,Salary Min,Salary Max,avg_salary,State,City
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,...,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",[R],"[Bachelor, Master]",NaN,137000,171000,154000,NY,New York
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,...,$1 to $2 billion (USD),-1,"[R, SQL]",[Bachelor],15.0,137000,171000,154000,VA,Chantilly
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,...,$100 to $500 million (USD),-1,"[Python, R, C++]",[PhD],NaN,137000,171000,154000,MA,Boston
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,...,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...","[Python, R, SQL]",[Bachelor],NaN,137000,171000,154000,MA,Newton
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,...,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee","[Python, R, SQL]",[],NaN,137000,171000,154000,NY,New York


In [350]:
df[['Size']].sample(5)

,Size
217,501 to 1000 employees
262,1001 to 5000 employees
195,51 to 200 employees
353,1 to 50 employees
646,5001 to 10000 employees


# My Approach to Standardizing Company Sizes

1. **Handling Missing or Invalid Values**:
   - I started by checking for any missing values or invalid inputs (like '-1'). If I found any, I returned `NaN` to keep the data clean.

2. **Processing Size Ranges**:
   - For sizes that indicate a range with a '+', such as '10000+', I converted them to a format like '10000-inf' to signify an unknown upper limit.

3. **Formatting Ranges**:
   - I also encountered sizes formatted as '51 to 200 employees.' I standardized these to a more uniform '51-200' format for easier comparison.

4. **Default Case**:
   - If a size didn’t match any of these patterns, I left it unchanged to preserve any valid entries.

5. **Applying the Function**:
   - Finally, I applied this function to the 'Size' column in the DataFrame, ensuring all company sizes are consistent and easy to analyze.


In [353]:
def standardize_size(size):
    if pd.isna(size) or size == '-1':  
        return np.nan
    if '+' in size:  
        return f"{size.split()[0]}+" 
    if 'to' in size:  
        return f"{size.split()[0]}-{size.split()[2]}" 
    return size

df['Size'] = df['Size'].apply(standardize_size)

In [355]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Revenue,Competitors,Skills,Education,Years_Experience,Salary Min,Salary Max,avg_salary,State,City
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001-5000,1993,Nonprofit Organization,...,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",[R],"[Bachelor, Master]",NaN,137000,171000,154000,NY,New York
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001-10000,1968,Company - Public,...,$1 to $2 billion (USD),-1,"[R, SQL]",[Bachelor],15.0,137000,171000,154000,VA,Chantilly
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001-5000,1981,Private Practice / Firm,...,$100 to $500 million (USD),-1,"[Python, R, C++]",[PhD],NaN,137000,171000,154000,MA,Boston
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501-1000,2000,Company - Public,...,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...","[Python, R, SQL]",[Bachelor],NaN,137000,171000,154000,MA,Newton
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51-200,1998,Company - Private,...,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee","[Python, R, SQL]",[],NaN,137000,171000,154000,NY,New York


# My Approach to Cleaning Founded Column

In [358]:
df[['Founded']].sample(5)

,Founded
622,2013
667,1989
314,2011
257,2010
23,1860


In [360]:
current_year = 2024
df['Age'] = current_year - df['Founded']
df['Age'] = df['Age'].replace(current_year + 1, np.nan)

In [362]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Competitors,Skills,Education,Years_Experience,Salary Min,Salary Max,avg_salary,State,City,Age
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001-5000,1993,Nonprofit Organization,...,"EmblemHealth, UnitedHealth Group, Aetna",[R],"[Bachelor, Master]",NaN,137000,171000,154000,NY,New York,31.0
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001-10000,1968,Company - Public,...,-1,"[R, SQL]",[Bachelor],15.0,137000,171000,154000,VA,Chantilly,56.0
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001-5000,1981,Private Practice / Firm,...,-1,"[Python, R, C++]",[PhD],NaN,137000,171000,154000,MA,Boston,43.0
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501-1000,2000,Company - Public,...,"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...","[Python, R, SQL]",[Bachelor],NaN,137000,171000,154000,MA,Newton,24.0
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51-200,1998,Company - Private,...,"Commerce Signals, Cardlytics, Yodlee","[Python, R, SQL]",[],NaN,137000,171000,154000,NY,New York,26.0


# My Approach to Cleaning Type of ownership Column

In [365]:
df[['Type of ownership']].sample(5)

,Type of ownership
103,Company - Public
305,Company - Private
570,Company - Private
551,Company - Private
127,Company - Private


In [367]:
df['Type of ownership'] = df['Type of ownership'].str.replace('Company - ', '')

In [369]:
df['Industry'] = df['Industry'].str.title()
df['Sector'] = df['Sector'].str.title()

# My Approach to Cleaning Revenue Data

1. **Handling Missing or Invalid Values**:
   - I started by checking for any missing values or invalid entries (like '-1'). If I found any, I returned `NaN` to ensure the dataset remains clean.

2. **Extracting Numerical Values**:
   - I used a regular expression to find all numerical values in the revenue string, as revenue can come in various formats.

3. **Identifying Units**:
   - I checked if the revenue mentioned 'million' or 'billion' to understand the scale. If it’s 'million', I multiplied the numbers by 1,000,000; if it’s 'billion', I multiplied by 1,000,000,000.

4. **Calculating Averages for Ranges**:
   - If I found two numbers (indicating a revenue range), I calculated the average. If only one number was found, I returned that value directly.

5. **Applying the Function**:
   - Finally, I applied this function to the 'Revenue' column, creating a new column called 'AverageRevenue' that standardizes revenue values for easier analysis.

In [372]:
df[['Revenue']].sample(5)

,Revenue
665,$50 to $100 million (USD)
493,$10+ billion (USD)
472,$10+ billion (USD)
561,$1 to $2 billion (USD)
652,$10 to $25 million (USD)


In [374]:
def clean_revenue(revenue):
    if pd.isna(revenue) or revenue == '-1':
        return np.nan 
    
    numbers = re.findall(r'\d+', revenue)
    
    if 'million' in revenue.lower():
        bounds = [float(num) * 1e6 for num in numbers]
    elif 'billion' in revenue.lower():
        bounds = [float(num) * 1e9 for num in numbers]
    else:
        return np.nan 
    
    if len(bounds) == 2:
        return np.mean(bounds)  
    elif len(bounds) == 1:
        return bounds[0]
    
    return np.nan  

df['AverageRevenue'] = df['Revenue'].apply(clean_revenue)

In [376]:
df[['AverageRevenue']].sample(5)

,AverageRevenue
368,NaN
574,300000000.0
86,NaN
222,300000000.0
88,NaN


# My Approach to Cleaning Compititors Data

In [379]:
df[['Competitors']].sample(5)

,Competitors
608,-1
234,-1
428,-1
614,Humana
602,"MIT Lincoln Laboratory, Lockheed Martin, North..."


In [381]:
df['Competitors'] = df['Competitors'].fillna('')

df['Number_of_Competitors'] = df['Competitors'].apply(lambda x: len(x.split(',')) if x else 0)

In [383]:
df = df.replace('Unknown', np.nan)

# My Approach to Cleaning Job title Data

In [386]:
df[['Job Title']].sample(5)

,Job Title
620,Data Scientist
617,Data Scientist
353,Data Scientist
561,Data Scientist Technical Specialist
187,Statistical Scientist


In [388]:
def get_experience_level(title):
    if 'senior' in title.lower() or 'lead' in title.lower():
        return 'Senior'  
    elif 'junior' in title.lower():
        return 'Junior'  
    else:
        return 'Mid-level' 

df['Experience_Level'] = df['Job Title'].apply(get_experience_level)

In [390]:
df['Rating'] = df['Rating'].astype('float')
df['Founded'] = df['Founded'].astype('int')
categorical_columns = ['Job Title', 'Company Name', 'Location', 'Type of ownership', 'Industry', 'Sector']
for col in categorical_columns:
    df[col] = df[col].astype('category') 

# Droping the unnecessary columns which have been cleaned

In [393]:
df.drop(['Job Description', 'Location', 'Headquarters', 'Revenue','Competitors'], axis=1, inplace=True)

# simplify job titles into standardized categories

In [396]:
def title_simplifier(title):
    title = title.lower()
    title_mappings = {
        'data scientist': 'data scientist',  
        'data engineer': 'data engineer',    
        'analyst': 'analyst',                
        'machine learning': 'mle',          
        'manager': 'manager',               
        'director': 'director',              
    }


    for key, simplified_title in title_mappings.items():
        if key in title:
            return simplified_title  
    return 'Other' 

In [398]:
df['Job Title'] = df['Job Title'].apply(title_simplifier)

In [400]:
labels = ['Low', 'Medium', 'High', 'Very High']   
df['SalaryCategory'] = pd.cut(df['avg_salary'], bins=4, labels=labels, right=False)

# Renaming columns in pandas DataFrame

In [403]:
df.rename(columns={
    'Job Title': 'JobTitle',
    'Salary Estimate': 'SalaryEstimate',
    'Rating': 'Rating',
    'Company Name': 'CompanyName',
    'Size': 'CompanySize',
    'Founded': 'FoundedYear',
    'Type of ownership': 'OwnershipType',
    'Industry': 'Industry',
    'Sector': 'Sector',
    'Skills': 'RequiredSkills',
    'Education': 'EducationLevel',
    'Years_Experience': 'YearsExperience',
    'Salary Min': 'SalaryMin',
    'Salary Max': 'SalaryMax',
    'avg_salary': 'AverageSalary',
    'State': 'State',
    'City': 'City',
    'Age': 'CompanyAge',
    'Revenue_Lower_Bound': 'RevenueLowerBound',
    'Number_of_Competitors': 'NumberOfCompetitors',
    'Experience_Level': 'ExperienceLevel'
}, inplace=True)

In [405]:
df.sample(10)

,JobTitle,SalaryEstimate,Rating,CompanyName,CompanySize,FoundedYear,OwnershipType,Industry,Sector,RequiredSkills,...,SalaryMin,SalaryMax,AverageSalary,State,City,CompanyAge,AverageRevenue,NumberOfCompetitors,ExperienceLevel,SalaryCategory
493,data scientist,95-119,3.5,Gap Inc.,10000++,1969,Public,"Department, Clothing, & Shoe Stores",Retail,"[Python, R, SQL]",...,95000,119000,107000,CA,San Francisco,55.0,1.000000e+10,3,Mid-level,Medium
67,data scientist,79-131,4.0,Grid Dynamics,1001-5000,2006,Private,Enterprise Software & Network Solutions,Information Technology,"[Python, R, SQL]",...,79000,131000,105000,CA,Santa Clara,18.0,7.500000e+07,3,Mid-level,Medium
400,analyst,110-163,4.5,Novetta,501-1000,2012,Private,Enterprise Software & Network Solutions,Information Technology,"[Python, R, SQL, Java, Scala]",...,110000,163000,136500,VA,Herndon,12.0,3.000000e+08,3,Senior,Medium
459,data scientist,69-116,-1.0,Blue Cloak LLC,NaN,-1,-1,-1,-1,"[Python, R, SQL, Java, Scala]",...,69000,116000,92500,VA,Sterling,NaN,NaN,1,Mid-level,Low
489,data scientist,95-119,3.5,Crossover Health,1-50,2010,Private,Health Care Services & Hospitals,Health Care,"[Python, R]",...,95000,119000,107000,CA,San Clemente,14.0,3.000000e+06,1,Mid-level,Medium
60,analyst,75-131,3.8,Swiss Re,10000++,1863,Public,Insurance Agencies & Brokerages,Insurance,"[Python, R, SQL]",...,75000,131000,103000,IN,Fort Wayne,161.0,1.000000e+10,3,Mid-level,Medium
620,data scientist,87-141,3.3,Cubic,5001-10000,1951,Public,Aerospace & Defense,Aerospace & Defense,"[Python, R, SQL, Scala]",...,87000,141000,114000,United States,United States,73.0,1.500000e+09,3,Mid-level,Medium
555,data scientist,128-201,-1.0,HireAi,NaN,-1,-1,-1,-1,"[Python, R, SQL, Java, C++]",...,128000,201000,164500,CA,San Francisco,NaN,NaN,1,Mid-level,High
134,mle,90-109,3.2,Triplebyte,51-200,2015,Private,Computer Hardware & Software,Information Technology,[R],...,90000,109000,99500,Remote,Remote,9.0,NaN,1,Mid-level,Low
646,data scientist,92-155,3.5,Maxar Technologies,5001-10000,-1,Public,Aerospace & Defense,Aerospace & Defense,"[Python, R, SQL, Java, C++]",...,92000,155000,123500,VA,Herndon,NaN,3.500000e+09,1,Mid-level,Medium


In [407]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   JobTitle             672 non-null    object  
 1   SalaryEstimate       672 non-null    object  
 2   Rating               672 non-null    float64 
 3   CompanyName          672 non-null    category
 4   CompanySize          628 non-null    object  
 5   FoundedYear          672 non-null    int32   
 6   OwnershipType        668 non-null    category
 7   Industry             672 non-null    category
 8   Sector               672 non-null    category
 9   RequiredSkills       672 non-null    object  
 10  EducationLevel       672 non-null    object  
 11  YearsExperience      444 non-null    float64 
 12  SalaryMin            672 non-null    int32   
 13  SalaryMax            672 non-null    int32   
 14  AverageSalary        672 non-null    int32   
 15  State                67

# Handling Missing Values

In [410]:
df.replace(-1, np.nan, inplace=True)
columns_with_nan = df.columns[df.isna().any()].tolist()

print("Columns with -1 or NaN values:", columns_with_nan)

Columns with -1 or NaN values: ['Rating', 'CompanySize', 'FoundedYear', 'OwnershipType', 'YearsExperience', 'CompanyAge', 'AverageRevenue']


In [412]:
df.isnull().sum()
missing_values = df.isnull().sum()
columns_with_missing = missing_values[missing_values > 0]
print(columns_with_missing)

Rating              50
CompanySize         44
FoundedYear        118
OwnershipType        4
YearsExperience    228
CompanyAge         118
AverageRevenue     240
dtype: int64


In [414]:
df['Rating'].fillna(df['Rating'].median(), inplace=True)

In [416]:
df['CompanySize'].value_counts()

CompanySize
51-200        135
1001-5000     104
1-50           86
201-500        85
10000++        80
501-1000       77
5001-10000     61
Name: count, dtype: int64

In [418]:
df['CompanySize'].fillna('Unknown', inplace=True)

In [420]:
df['FoundedYear'].fillna(df['FoundedYear'].median(), inplace=True)

In [422]:
# Drop rows with missing values in the OwnershipType column
df.dropna(subset=['OwnershipType'], inplace=True)

In [424]:
knn_imputer = KNNImputer(n_neighbors=5)
df[['YearsExperience', 'AverageRevenue']] = knn_imputer.fit_transform(df[['YearsExperience', 'AverageRevenue']])

In [426]:
df['CompanyAge'].fillna(2024 - df['FoundedYear'], inplace=True)

In [428]:
df['OwnershipType'].replace('-1', df['OwnershipType'].mode()[0], inplace=True)
df['Sector'].replace('-1', df['Sector'].mode()[0], inplace=True)

In [430]:
# Replace '-1' in 'Industry' using loc
df.loc[df['Industry'] == '-1', 'Industry'] = np.nan
df['Industry'].fillna(df['Industry'].mode()[0], inplace=True)

In [432]:
df.isnull().sum()

JobTitle               0
SalaryEstimate         0
Rating                 0
CompanyName            0
CompanySize            0
FoundedYear            0
OwnershipType          0
Industry               0
Sector                 0
RequiredSkills         0
EducationLevel         0
YearsExperience        0
SalaryMin              0
SalaryMax              0
AverageSalary          0
State                  0
City                   0
CompanyAge             0
AverageRevenue         0
NumberOfCompetitors    0
ExperienceLevel        0
SalaryCategory         0
dtype: int64

In [434]:
df.describe()

,Rating,FoundedYear,YearsExperience,SalaryMin,SalaryMax,AverageSalary,CompanyAge,AverageRevenue,NumberOfCompetitors
count,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000,6.680000e+02,668.000000
mean,3.874401,1986.646707,6.101598,99190.119760,148125.748503,123657.934132,37.353293,2.518344e+09,1.449102
std,0.588129,37.439179,10.660060,33091.369914,48117.195714,39661.944569,37.439179,2.976148e+09,0.807823
min,2.000000,1781.000000,1.000000,31000.000000,56000.000000,43500.000000,5.000000,1.000000e+06,1.000000
25%,3.500000,1983.000000,3.000000,79000.000000,119000.000000,103000.000000,15.000000,2.066250e+08,1.000000
50%,3.800000,1999.000000,5.000000,91000.000000,133000.000000,114000.000000,25.000000,2.060000e+09,1.000000
75%,4.300000,2009.000000,6.256236,122000.000000,165000.000000,136500.000000,41.000000,3.500000e+09,1.000000
max,5.000000,2019.000000,160.000000,212000.000000,331000.000000,271500.000000,243.000000,1.000000e+10,3.000000


In [436]:
df.sample(5)

,JobTitle,SalaryEstimate,Rating,CompanyName,CompanySize,FoundedYear,OwnershipType,Industry,Sector,RequiredSkills,...,SalaryMin,SalaryMax,AverageSalary,State,City,CompanyAge,AverageRevenue,NumberOfCompetitors,ExperienceLevel,SalaryCategory
65,data engineer,79-131,3.7,Bayview Asset Management,1001-5000,1995.0,Private,Investment Banking & Asset Management,Finance,"[Python, R, SQL, C++]",...,79000,131000,105000,FL,Coral Gables,29.0,3.000000e+08,1,Mid-level,Medium
144,mle,90-109,5.0,"Expedition Technology, Inc.",1-50,2013.0,Private,Aerospace & Defense,Aerospace & Defense,"[Python, R, C++, Julia]",...,90000,109000,99500,VA,Herndon,11.0,1.750000e+07,1,Mid-level,Low
528,data scientist,212-331,3.5,Aptive,51-200,2012.0,Private,Consulting,Business Services,"[Python, R, SQL, Java, C++]",...,212000,331000,271500,DC,Washington,12.0,3.750000e+07,1,Mid-level,Very High
121,data engineer,99-132,4.7,Scoop,51-200,2015.0,Private,Internet,Information Technology,"[Python, R, Scala]",...,99000,132000,115500,CA,San Francisco,9.0,2.560600e+09,1,Senior,Medium
614,data scientist,87-141,3.2,First Health Group,10000++,1853.0,Private,Insurance Carriers,Insurance,[R],...,87000,141000,114000,MA,Wellesley,171.0,1.000000e+10,1,Mid-level,Medium
